<img style="float:left" width="70%" src="pics/escudo_COLOR_1L_DCHA.png">
<img style="float:right" width="15%" src="pics/PythonLogo.svg">
<br style="clear:both;">


# *Wumpus*
### *Sistemas Inteligentes* (Curso 2024-2025)
#### Clase semana 5


<h2 style="display: inline-block; padding: 4mm; padding-left: 2em; background-color: navy; line-height: 1.3em; color: white; border-radius: 10px;">Hacia la versión "manual" del juego - II</h2>

## Docentes

 - Pedro Latorre Carmona

---
Los objetivos de la clase de esta semana van a ser (idealmente) los siguientes:

1. Colocar de forma aleatoria, en un tablero de $6\times6$:

    - Dos casillas, con **huecos**, 
    - El **wumpus**,
    - El **oro**


2. Restringir la distancia del **oro** al **agente**, de tal forma que no pueda estar uno al lado del otro.


3. Generar los vecinos en las posiciones: $(i-1,j)$, $(i+1,j)$, $(i,j-1)$, $(i,j+1)$ correspondientes, que tengan sentido, a partir de la disposición de las casillas en el apartado anterior.


4. Crear una función de mérito que no solamente tenga en cuenta la cercanía a la casilla de **oro**, sino a las casillas de **hedor**, o **brisa**.


5. **OPCIONAL**: Crear una función que impida el movimiento diagonal entre casillas.

In [1]:
import random
import math
from copy import deepcopy
from typing import List, Tuple


In [2]:
#definicion de los elementos como numeros
BLANCO = 0
AGENTE = 1
HOYO = 2
WUMPUS = 3
ORO = 4
HEDOR = 5
BRISA = 6
BRISA_HEDOR =9
BRISA_ORO=8
HEDOR_ORO=7

In [3]:
class Tablerowumpus:

    def __init__(self, matrix: List[List[int]]):
        self.tamano = 6

        # Validar que la matriz tenga el tamaño correcto
        if not self.es_matriz_valida(matrix):
            raise ValueError(f"La matriz proporcionada debe ser de tamaño {self.tamano}x{self.tamano}.")

        self.matrix = deepcopy(matrix)
        self.pos_agente = (5, 0)  # Posición fija del agente

        # Colocar el agente en la matriz
        self.placeTile(self.pos_agente[0], self.pos_agente[1], AGENTE)

        # Colocar los demás elementos del juego
        self.colocar_elementos()

    def __eq__(self, other) -> bool:
        for i in range(self.tamano):
            for j in range(self.tamano):
                if self.matrix[i][j] != other.matrix[i][j]:
                    return False
        return True

    def setMatrix(self, matrix: List[List[int]]):
        if not self.es_matriz_valida(matrix):
            raise ValueError(f"La matriz proporcionada debe ser de tamaño {self.tamano}x{self.tamano}.")
        self.matrix = deepcopy(matrix)

    def getMatrix(self) -> List[List[int]]:
        return deepcopy(self.matrix)

    def placeTile(self, row: int, col: int, tile: int):
        if 0 <= row < self.tamano and 0 <= col < self.tamano:
            self.matrix[row][col] = tile
        else:
            raise IndexError(f"Las coordenadas ({row}, {col}) están fuera de los límites del tablero.")

    def colocar_elementos(self):
        posiciones_disponibles = [(i, j) for i in range(self.tamano) for j in range(self.tamano)]
        # Remover la posición del agente de las disponibles
        if self.pos_agente in posiciones_disponibles:
            posiciones_disponibles.remove(self.pos_agente)

        # Colocar el Wumpus
        wumpus_pos = random.choice(posiciones_disponibles)
        self.placeTile(wumpus_pos[0], wumpus_pos[1], WUMPUS)
        posiciones_disponibles.remove(wumpus_pos)

        # Colocar el oro asegurando que no esté adyacente al agente
        posiciones_no_adyacentes = [pos for pos in posiciones_disponibles if
                                     not self.es_adyacente(pos, self.pos_agente)]
        if not posiciones_no_adyacentes:
            raise ValueError("No hay posiciones disponibles para colocar el oro que no estén adyacentes al agente.")
        oro_pos = random.choice(posiciones_no_adyacentes)
        self.placeTile(oro_pos[0], oro_pos[1], ORO)
        posiciones_disponibles.remove(oro_pos)

        # Colocar los 2 huecos
        if len(posiciones_disponibles) < 2:
            raise ValueError("No hay suficientes posiciones disponibles para colocar los huecos.")
        hoyos_pos1 = random.choice(posiciones_disponibles)
        self.placeTile(hoyos_pos1[0], hoyos_pos1[1], HOYO)
        posiciones_disponibles.remove(hoyos_pos1)

        hoyos_pos2 = random.choice(posiciones_disponibles)
        self.placeTile(hoyos_pos2[0], hoyos_pos2[1], HOYO)
        posiciones_disponibles.remove(hoyos_pos2)

        # Opcional: Colocar casillas de hedor alrededor del Wumpus
        vecinos_wumpus = self.obtener_vecinos(wumpus_pos)
        for vec in vecinos_wumpus:
            if self.matrix[vec[0]][vec[1]] == BLANCO:
                self.placeTile(vec[0], vec[1], HEDOR)
            elif self.matrix[vec[0]][vec[1]] == ORO:
                self.placeTile(vec[0], vec[1], HEDOR_ORO)

        # Colocar casillas de brisa alrededor de los huecos
        for hoyo_pos in [hoyos_pos1, hoyos_pos2]:
            vecinos_hoyo = self.obtener_vecinos(hoyo_pos)
            for vec in vecinos_hoyo:
                if self.matrix[vec[0]][vec[1]] == BLANCO:
                    self.placeTile(vec[0], vec[1], BRISA)
                elif self.matrix[vec[0]][vec[1]] == HEDOR:
                    self.placeTile(vec[0], vec[1], BRISA_HEDOR)
                elif self.matrix[vec[0]][vec[1]] == ORO:
                    self.placeTile(vec[0], vec[1], BRISA_ORO)

    def es_matriz_valida(self, matrix: List[List[int]]) -> bool:
        if len(matrix) != self.tamano:
            return False
        for fila in matrix:
            if len(fila) != self.tamano:
                return False
            for casilla in fila:
                if casilla not in {BLANCO, AGENTE, HOYO, WUMPUS, ORO, HEDOR, BRISA, BRISA_HEDOR, BRISA_ORO, HEDOR_ORO}:
                    return False
        return True

    def es_adyacente(self, pos1: Tuple[int, int], pos2: Tuple[int, int]) -> bool:
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1]) == 1

    def obtener_vecinos(self, pos: Tuple[int, int]) -> List[Tuple[int, int]]:
        i, j = pos
        vecinos = []
        posibles = [(i - 1, j), (i + 1, j), (i, j - 1), (i, j + 1)]
        for x, y in posibles:
            if 0 <= x < self.tamano and 0 <= y < self.tamano:
                vecinos.append((x, y))
        return vecinos

    def utility(self) -> float:
        posAgente = [0, 0]
        posOro = [0, 0]
        alto = len(self.matrix)
        ancho = len(self.matrix[0])

        for i in range(alto):
            for j in range(ancho):
                if self.matrix[i][j] == AGENTE:
                    posAgente = [i, j]
                    print("Posición del Agente encontrada:", posAgente)
                if self.matrix[i][j] in [ORO, BRISA_ORO, HEDOR_ORO]:
                    posOro = [i, j]
                    print("Posición del Oro encontrada:", posOro)

        distancia = math.sqrt((posOro[0] - posAgente[0]) ** 2 + (posOro[1] - posAgente[1]) ** 2)
        print(f"Distancia Euclidiana entre Agente y Oro: {distancia}")

        # Obtener los vecinos del agente
        vecinos_agente = self.obtener_vecinos(tuple(posAgente))
        print(f"Vecinos del Agente: {vecinos_agente}")

        # Contar las casillas de penalización
        contador_penalizaciones = {
            HEDOR: 0,
            BRISA: 0,
            BRISA_HEDOR: 0,
            BRISA_ORO: 0,
            HEDOR_ORO: 0
        }

        for vec in vecinos_agente:
            tile = self.matrix[vec[0]][vec[1]]
            if tile in contador_penalizaciones:
                contador_penalizaciones[tile] += 1

        print("Conteo de casillas de penalización cerca del Agente:", contador_penalizaciones)

        # Definir pesos de penalización
        pesos_penalizacion = {
            HEDOR: 0.1,
            BRISA: 0.1,
            BRISA_HEDOR: 0.2,
            BRISA_ORO: 0.15,
            HEDOR_ORO: 0.15
        }

        # Calcular penalización total
        penalizacion_total = 0.0
        for tile, count in contador_penalizaciones.items():
            penalizacion_total += pesos_penalizacion.get(tile, 0) * count

        print(f"Penalización Total: {penalizacion_total}")

        # Calcular la utilidad final
        utilidad = (1 / (distancia + 1e-2)) - penalizacion_total
        print(f"Utilidad Calculada: {utilidad}")

        return utilidad

    def imprimir_tablero(self):
        for fila in self.matrix:
            print(' '.join(str(casilla) for casilla in fila))
        print()

In [4]:
if __name__ == "__main__":
    # Crear una matriz de 6x6 llena de BLANCO (0)
    matriz_inicial = [[BLANCO for _ in range(6)] for _ in range(6)]

    # Crear una instancia de Tablerowumpus pasando la matriz inicial
    tablero = Tablerowumpus(matriz_inicial)
    tablero.imprimir_tablero()
    print("Utilidad del tablero:", tablero.utility())

0 0 0 0 0 0
0 0 0 0 0 0
6 0 0 4 0 0
2 6 0 0 0 0
6 0 5 6 0 0
1 5 3 2 6 0

Posición del Oro encontrada: [2, 3]
Posición del Agente encontrada: [5, 0]
Distancia Euclidiana entre Agente y Oro: 4.242640687119285
Vecinos del Agente: [(4, 0), (5, 1)]
Conteo de casillas de penalización cerca del Agente: {5: 1, 6: 1, 9: 0, 8: 0, 7: 0}
Penalización Total: 0.2
Utilidad Calculada: 0.035148011217800396
Utilidad del tablero: 0.035148011217800396
